importo i pacchetti necessari

In [1]:
import pandas as pd
import requests
import coordinates_calcolator as cc
import url_contructor as uc
import image_visualizator as iv
from db_manager import create_empty_csv, populate_csv
import toml

prendo i dati dal csv filtrato

In [2]:
mars_data = pd.read_csv('Mars_crater_db_filtered.csv')
zoom = 6
#create_empty_csv('Mars_crater_db_images.csv', ['id', 'image'])
toml_data = toml.load('secrets.toml')
api_key = toml_data['api_key']

dato un valore di inizio e uno di fine da mars data faccio partire una request costruita con url_contructor con lat e lon da coordinates_calcolator

In [3]:
def get_data(start, end, api_key):
    data = []
    for i in range(start, end):
        lat = mars_data['lat'][i]
        #print(lat)
        lon = mars_data['lon'][i]
        #print(lon)
        id = mars_data['id'][i]
        #print(id)
        lat, lon = cc.lat_lon_to_tile_cil(lat, lon, zoom)
        #print (lat, lon)
        url = uc.get_request_url(lat, lon, tileMatrix=zoom)
        print(url)
        params = {'api_key': api_key}  # Aggiungi la chiave API come parametro
        
        # Esegui la richiesta HTTP con la chiave API
        response = requests.get(url, params=params)
       
        if response.status_code == 200:
            data.append([id,response.content])
        else:
            print('error')
    return data

In [4]:
data = get_data(0,20, api_key)



for i in range(len(data)):
    id = data[i][0]
    print(id)
    image = data[i][1]
    iv.image_visualizator_from_bin(image)


popolo il csv 'Mars_crater_db_images.csv' con i dati ottenuti

In [8]:
populate_csv(data)


In [6]:
print(len(data))

def build_mars_url(x, y, z, p, d='', locale='', bbox='', extent='', sfz=''):
    base_url = "https://trek.nasa.gov/mars/"
    params = {
        'v': '0.1',
        'x': x,
        'y': y,
        'z': z,
        'p': p,
        'd': d,
        'locale': locale,
        'b': bbox,
        'e': extent,
        'sfz': sfz,
        'w': ''  # Il parametro 'w' è vuoto nell'URL originale
    }
    # Costruisci l'URL aggiungendo i parametri come stringa di query
    url = base_url + '?' + '&'.join([f"{key}={value}" for key, value in params.items()])
    return url

def get_mars_data(x, y, z, p, d='', locale='', bbox='', extent='', sfz=''):
    # Costruisci l'URL utilizzando la funzione build_mars_url
    url = build_mars_url(x, y, z, p, d, locale, bbox, extent, sfz)
    print(url)
    try:
        # Esegui la richiesta GET all'URL
        response = requests.get(url)
        # Verifica se la richiesta è andata a buon fine (status code 200)
        if response.status_code == 200:
            # Restituisci il contenuto della risposta
            return response.content
        else:
            # Se la richiesta non è andata a buon fine, stampa un messaggio di errore
            print(f"Errore durante la richiesta GET: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        # Se si verifica un'eccezione durante la richiesta, stampa l'errore
        print(f"Errore durante la richiesta GET: {e}")
        return None

In [5]:
def get_data_from_coordinates(lat,lon,zoom, api_key):
    data = []
    lat, lon = cc.lat_lon_to_tile_cil(lat, lon, zoom)
    url = uc.get_request_url(lat, lon, tileMatrix=zoom)
    print(url)
    params = {'api_key': api_key}  # Aggiungi la chiave API come parametro
    
    # Esegui la richiesta HTTP con la chiave API
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data.append([id,response.content])
    else:
        print('error')
    return data

In [17]:
data_new = get_data_from_coordinates(16.8007, -4.8254,0, api_key)
